<a href="https://colab.research.google.com/github/Svein-Tore/colab/blob/main/pi_regulator_niva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installer ved behov:
!pip install control matplotlib numpy ipywidgets

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import control as ctrl
from ipywidgets import interact, FloatSlider, ToggleButton

# =========================
# PI-regulator: SIMC eller ZN
# =========================
def pi_controller(Kp_process, T, L, lam, ZN=False):
    s = ctrl.TransferFunction.s

    if ZN:
        # Ziegler–Nichols (åpen sløyfe / reaksjonskurve)
        Kc = 0.9 * T / (Kp_process * L)
        Ti = 3 * L
        tuning = "Ziegler–Nichols"
    else:
        # SIMC PI
        Kc = T / (Kp_process * (lam + L))
        Ti = T
        tuning = "SIMC"

    C = Kc * (1 + 1/(Ti*s))
    return C, Kc, Ti, tuning


# =========================
# FOPDT-prosess
# =========================
def fopdt_system(K, T, L):
    num_pade, den_pade = ctrl.pade(L, 1)
    dead_time = ctrl.TransferFunction(num_pade, den_pade)
    G_no_delay = ctrl.TransferFunction([K], [T, 1])
    return G_no_delay * dead_time


# =========================
# Plot-funksjon med y0
# =========================
def plot_step_response(K=1.0, T=97.0, L=6.0, A=20.0, y0=5.0, lam=48.5, ZN=False):

    G = fopdt_system(K, T, L)
    C, Kc, Ti, tuning = pi_controller(K, T, L, lam, ZN)

    # Lukket sløyfe
    T_closed = ctrl.feedback(C * G, 1)

    # Simuleringstid
    t_end =max(5*(lam + L), 5*T)
    t = np.linspace(0, t_end, 1500)

    # Steg med amplitude A og startnivå y0
    t, y = ctrl.step_response(T_closed * A, t)
    y = y0 + y  # forskyv kurven med y0

    plt.figure(figsize=(10, 5))
    plt.plot(t, y)
    plt.title(f"Steprespons – {tuning} PI")
    plt.xlabel("Tid (s)")
    plt.ylabel("y(t)")
    plt.grid(True)

    # Vis regulatorparametere
    plt.text(0.7,0.2 ,
        f"Tuning: {tuning}\nKc = {Kc:.3f}\nTi = {Ti:.2f} s\nStartnivå y0 = {y0:.2f}",
        transform=plt.gca().transAxes,
        bbox=dict(facecolor='white', alpha=0.85)
    )

    plt.show()


# =========================
# Interaktive kontroller
# =========================
interact(
    plot_step_response,
    K=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='K'),
    T=FloatSlider(value=97.0, min=1.0, max=200.0, step=1.0, description='T'),
    L=FloatSlider(value=6.0, min=0.1, max=30.0, step=0.1, description='L'),
    A=FloatSlider(value=20.0, min=0.1, max=100.0, step=0.1, description='A'),
    y0=FloatSlider(value=5.0, min=0.0, max=100.0, step=0.1, description='y0'),
    lam=FloatSlider(value=97*0.5, min=0.3*97, max=2*97, step=1.0, description='λ'),
    ZN=ToggleButton(value=False, description='Endre tuning')
)